In [41]:
import sys ; sys.path.append("../")
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import cvlib
import time

%matplotlib inline
%load_ext cython

In [44]:
img = cv.imread('shop.jpeg', cv.IMREAD_COLOR)
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
kernel = np.array([[-1, -1, -1], [-1, 9.5, -1], [-1, -1, -1]])
#img_sharpen = sharpen(img=img, kernel=kernel)

### Caso incial

In [43]:
%%cython -a
import numpy as np
def sharpen(img:np.array, kernel:np.array):
    
    # Get the height, width, and number of channels of the image
    height,width,c =img.shape[0],img.shape[1],img.shape[2]
    
    # Get the height, width, and number of channels of the kernel
    kernel_height,kernel_width = kernel.shape[0],kernel.shape[1]
    
    # Create a new image of original img size minus the border 
    # where the convolution can't be applied
    new_img = np.zeros((height-kernel_height+1,width-kernel_width+1,3)) 
    
    # Loop through each pixel in the image
    # But skip the outer edges of the image
    for i in range(kernel_height//2, height-kernel_height//2-1):
        for j in range(kernel_width//2, width-kernel_width//2-1):
            # Extract a window of pixels around the current pixel
            window = img[i-kernel_height//2 : i+kernel_height//2+1,j-kernel_width//2 : j+kernel_width//2+1]
            # Apply the convolution to the window and set the result as the value of the current pixel in the new image
            var = np.absolute(int((window[:,:,0] * kernel).sum()))
            if (var > 255):
                new_img[i, j, 0] = int(255)
            else:
                new_img[i, j, 0] = var
            var = np.absolute(int((window[:,:,1] * kernel).sum()))
            if (var > 255):
                new_img[i, j, 1] = int(255)
            else:
                new_img[i, j, 1] = var
            var = np.absolute(int((window[:,:,2] * kernel).sum()))
            if (var > 255):
                new_img[i, j, 2] = int(255)
            else:
                new_img[i, j, 2] = var
    return new_img.astype(np.uint8)

Content of stderr:
/Users/carlosalvarado/.cache/ipython/cython/_cython_magic_d6e4f6026066c1d6fbbb321efb8e2f7869d8409b.c:2498:3: warning: code will never be executed [-Wunreachable-code]
  goto __pyx_L3_error;
  ^~~~~~~~~~~~~~~~~~~
/Users/carlosalvarado/.cache/ipython/cython/_cython_magic_d6e4f6026066c1d6fbbb321efb8e2f7869d8409b.c:4732:21: warning: fallthrough annotation in unreachable code [-Wunreachable-code-fallthrough]
                    CYTHON_FALLTHROUGH;
                    ^
/Users/carlosalvarado/.cache/ipython/cython/_cython_magic_d6e4f6026066c1d6fbbb321efb8e2f7869d8409b.c:530:34: note: expanded from macro 'CYTHON_FALLTHROUGH'
      #define CYTHON_FALLTHROUGH __attribute__((fallthrough))
                                 ^
/Users/carlosalvarado/.cache/ipython/cython/_cython_magic_d6e4f6026066c1d6fbbb321efb8e2f7869d8409b.c:4743:21: warning: fallthrough annotation in unreachable code [-Wunreachable-code-fallthrough]
                    CYTHON_FALLTHROUGH;
                    ^
/U

In [45]:
%timeit sharpen(img, kernel)

3.62 s ± 19.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Caso 2

In [50]:
%%cython -a
import cython
import numpy as np

@cython.boundscheck(False)
cpdef unsigned char[:, :] sharpen_cython(unsigned char[:, :] img, unsigned char[:, :] kernel):    
    # Get the height, width, and number of channels of the image
    height,width,c =img.shape[0],img.shape[1],img.shape[2]
    
    # Get the height, width, and number of channels of the kernel
    kernel_height,kernel_width = kernel.shape[0],kernel.shape[1]
    
    # Create a new image of original img size minus the border 
    # where the convolution can't be applied
    new_img = np.zeros((height-kernel_height+1,width-kernel_width+1,3)) 
    
    # Loop through each pixel in the image
    # But skip the outer edges of the image
    for i in range(kernel_height//2, height-kernel_height//2-1):
        for j in range(kernel_width//2, width-kernel_width//2-1):
            # Extract a window of pixels around the current pixel
            window = img[i-kernel_height//2 : i+kernel_height//2+1,j-kernel_width//2 : j+kernel_width//2+1]
            # Apply the convolution to the window and set the result as the value of the current pixel in the new image
            var = np.absolute(int((window[:,:,0] * kernel).sum()))
            if (var > 255):
                new_img[i, j, 0] = int(255)
            else:
                new_img[i, j, 0] = var
            var = np.absolute(int((window[:,:,1] * kernel).sum()))
            if (var > 255):
                new_img[i, j, 1] = int(255)
            else:
                new_img[i, j, 1] = var
            var = np.absolute(int((window[:,:,2] * kernel).sum()))
            if (var > 255):
                new_img[i, j, 2] = int(255)
            else:
                new_img[i, j, 2] = var
    return new_img.astype(np.uint8)


Error compiling Cython file:
------------------------------------------------------------
...
    for i in range(kernel_height//2, height-kernel_height//2-1):
        for j in range(kernel_width//2, width-kernel_width//2-1):
            # Extract a window of pixels around the current pixel
            window = img[i-kernel_height//2 : i+kernel_height//2+1,j-kernel_width//2 : j+kernel_width//2+1]
            # Apply the convolution to the window and set the result as the value of the current pixel in the new image
            var = np.absolute(int((window[:,:,0] * kernel).sum()))
                                              ^
------------------------------------------------------------

/Users/carlosalvarado/.cache/ipython/cython/_cython_magic_47ed0af7e2cfac8421c9bcee0189b4651beb22ad.pyx:23:46: Too many indices specified for type unsigned char[:, :]

Error compiling Cython file:
------------------------------------------------------------
...
            var = np.absolute(int((window[

In [ ]:
%timeit sharpen(img, kernel)